In [ ]:
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


def readPdf(list_file):
    """
    Fonction qui transforme des pdf en texte :

    Args:
        file (list): liste des fichiers

    Returns:
        text : contenu du fichier
    """
    list_texts = []
    for file in list_file:
        reader = PdfReader(file)
        nb_pages = len(reader.pages)
        for i in range(nb_pages):
            page = reader.pages[i]
            text = page.extract_text()
            list_texts.append(text)
    return list_texts



document = readPdf(["pdf-exemple.pdf","sample.pdf","rugby.pdf","mes-fiches-animaux-de-la-ferme.pdf","vaches.pdf"])
#document = readPdf([])

docs = [Document(page_content=page_text) for page_text  in document]
#Fait une découpe plus intelligente que ma fonction grâce à chunk_overlap qui va essayer de trouver une phrase avant la découpe.
splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap=75,separators=["\n\n","\n",".","!","?"])
splits_docs = splitter.split_documents(docs)
chunks = [doc.page_content for doc in splits_docs]

#Nettoyage en remplaçant sauf ligne par espace : 
def clean_chunk(text):
    return ' '.join(text.replace('\n', ' ').split())

chunks = [clean_chunk(chunk) for chunk in chunks]


In [34]:
#On analyse nos chunks : 
for i, chunk in enumerate(chunks[:10]):
    print(f"Chunk {i} :\n{chunk}\n{'-'*50}")


Chunk 0 :
Fichier PDF d'exemple Le Portable Document Format (qui se traduit de l'anglais en « format de document portable »), généralement abrégé PDF, est un format de fichier informatique créé par Adobe Systems. C'est un format ouvert dont les spécifications sont publiques et utilisables librement (certains éléments sont à disposition sur le site Adobe). Il est
--------------------------------------------------
Chunk 1 :
dérivé du format PostScript et contient des données au format XML. Le format PDF est un format de fichier qui préserve les polices, les images, les objets graphiques et la mise en forme de tout document source, quelles que soient l'application et la plate-forme utilisées pour le créer. Les fichiers PDF peuvent être créés avec des
--------------------------------------------------
Chunk 2 :
options personnalisées, tant aux niveaux de la compression des images et des textes, de la qualité d'impression du fichier, ainsi que du verrouillage (interdiction d'impression, de 

In [ ]:

#On embedding (transférer les phrases par des vecteurs)
modelEmbedding = SentenceTransformer("all-MiniLM-L6-v2")
embedding = modelEmbedding.encode(chunks, show_progress_bar=True, convert_to_numpy=True)

user_text = input("Entrez votre phrase : ")
embedding_user = modelEmbedding.encode([user_text],show_progress_bar=True, convert_to_numpy=True).astype("float32")


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import faiss
import math

dimension = embedding.shape[1]
index = faiss.IndexFlatIP(dimension)  # 32 = nombre de voisins (standard)
output_text = []

#on ajoute les embeddings : 
index.add(embedding)
print(f"Question de l'utilisateur : {user_text}\n")
distances, indices = index.search(embedding_user, k=4)
for i, idx in enumerate(indices[0]):
    score = math.ceil(distances[0][i]*100)/100
    print(f"{i+1}. Score = {distances[0][i]:.2f} | {chunks[idx]}")
    output_text.append(chunks[idx])

Question de l'utilisateur : Comment vont les joueurs de Rugby ? ca va ?

1. Score = 0.71 | Les règle du rugby La durée:la durée d'un match de rugby est de 2x 40minutes pour les séniors. les règles: il doit y avoir 15 joueurs obligatoirement dans une équipe de rugby. Un match commence par un coup d’envoi. Après le coup d’envoi, tout joueur qui est en jeu peut se saisir du ballon et courir en le portant. Un joueur peut lancer le ballon ou le botter.
2. Score = 0.66 | Un joueur peut faire un touché à terre dans un en-but. Un porteur du ballon peut effectuer un raffut sur un adversaire. Toute action d’un joueur doit être faite dans le respect des Règles du Jeu. Un joueur et obliger de passer la balle en arrière Un terrain de rugby:
3. Score = 0.64 | • Hors-jeu :quand on et en avant du ballon. • Mêlée :huit joueurs de chaque équipe, liés entre eux sur trois lignes pour chaque équipe. • Pénalité :La pénalité est un geste technique. • Touch e: il faut lancer le ballons a la quand on et en tou

In [37]:
"""
#On utilise la recherche sémantique pour retrouver les vecteurs les plus proches (donc les plus probables)
result = util.semantic_search(embedding_user,embedding)[0]
print(result)
output_text = []

#On inclut une réponse par défaut si la similarité est pas assez haute pour répondre à la question posée.

for item in result:
    score = math.ceil(item["score"]*100)/100
    print(score, "|", chunks[item["corpus_id"]])
    
    #On fait un top 10 des réponses les plus probables et qui sont le plus probable de sortir (arrondi >0.6)
    if len(output_text) < 10 and score >= 0.6:
        output_text.append(chunks[item["corpus_id"]])
"""

'\n#On utilise la recherche sémantique pour retrouver les vecteurs les plus proches (donc les plus probables)\nresult = util.semantic_search(embedding_user,embedding)[0]\nprint(result)\noutput_text = []\n\n#On inclut une réponse par défaut si la similarité est pas assez haute pour répondre à la question posée.\n\nfor item in result:\n    score = math.ceil(item["score"]*100)/100\n    print(score, "|", chunks[item["corpus_id"]])\n\n    #On fait un top 10 des réponses les plus probables et qui sont le plus probable de sortir (arrondi >0.6)\n    if len(output_text) < 10 and score >= 0.6:\n        output_text.append(chunks[item["corpus_id"]])\n'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM
import time
from datetime import datetime
from csv import writer

t0 = time.time()
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "google/flan-t5-large" Très très bon déjà mais limite 512 tokens
#model_name = "google/flan-t5-xl"
#
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
DEFAULT_RESPONSE = "Désolé, les fichiers ne permettent pas de répondre à vos questions."

prompt_template = """
Voici des extraits de documents jugés pertinents pour répondre à la question ci-dessous. Reformule ta réponse de manière claire.
Extraits : 
""" + str(output_text) +  """\n\n
Consignes : \n
- Ne génère ta réponse qu'en te basant strictement sur le contenu des documents ci-dessus, en la reformulant pour être compréhensible.
- Si les documents ne permettent pas de répondre clairement à la question, dis simplement : 
"Je ne peux pas répondre à cette question avec les documents fournis."
Question :
"""+ user_text  +"""
Réponse:
"""


if len(output_text) > 0 and len(document) > 0:
    #On remplace les mots par des vecteurs : 
    inputs = tokenizer(prompt_template,return_tensors="pt")
    print("Longueur token du prompt : ",inputs['input_ids'].shape[1])
    eos_token_id = tokenizer.eos_token_id  # Utilise le token EOS pour arrêter la génération

    # Permet de faire répondre le modèle sur les vecteurs que l'on vient de créer
    outputs = model.generate(**inputs, temperature=0.1, max_new_tokens=200, eos_token_id=eos_token_id)

    #On décode la réponse renvoyée par le modèle : 
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    print(response)
    
    #Pour les stats :
    result_response = "Réponse correcte"
elif len(document) == 0:
    prompt_template = """
    Répond à la question :""" + user_text
    
    #On remplace les mots par des vecteurs : 
    inputs = tokenizer(prompt_template,return_tensors="pt")
    print("Longueur token du prompt : ",tokenizer['input_ids'].shape[1])
    eos_token_id = tokenizer.eos_token_id  # Utilise le token EOS pour arrêter la génération

    # Permet de faire répondre le modèle sur les vecteurs que l'on vient de créer
    outputs = model.generate(**inputs, temperature=0.1, max_new_tokens=500, eos_token_id=eos_token_id)

    #On décode la réponse renvoyée par le modèle : 
    response = tokenizer.decode(outputs[0],skip_special_tokens=True) + "\n⚠️ CETTE REPONSE A ETE FOURNI SANS DOCUMENT ANNEXE, ELLE EST DONC GENERALE ⚠️"
    print(response)
    
    #Pour les stats :
    result_response = "Réponse hors-sujet sans fichier"
else:
    #Si il n'y a aucune réponse assez similaire (seuil 0.6)
    response = DEFAULT_RESPONSE
    result_response = "Hors-sujet"
t1 = time.time()
total_time = round(t1-t0,5)

#Inscription de la date : 
date_actuelle = datetime.now().strftime("%d/%m, %H:%M:%S")
# Inscription dans le fichier txt : 
with open("data_saved_v2.txt", "a", newline="", encoding="utf-8") as fichier:
    fichier.write("-------------------------------------------------------------------\n")
    fichier.write("Test du : " + date_actuelle + " avec modèle : " + model_name + "\n")
    fichier.write("-------------------------------------------------------------------\n")
    fichier.write("Temps d'éxecution : " + str(total_time) + "\n")
    fichier.write("Question posée : " + user_text + "\n")
    fichier.write("Réponse apportée : " + response + "\n")
    fichier.write("===================================================================\n\n")
fichier.close()



#Pareil mais en csv : 
dictio_csv = [model_name,date_actuelle,total_time,result_response]
with open("data_saved.csv","a",encoding="utf-8",newline="") as fichier:
    w = writer(fichier)
    w.writerow(dictio_csv)
    fichier.close()




Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Longueur token du prompt :  597


c:\Users\PC7\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Voici des extraits de documents jugés pertinents pour répondre à la question ci-dessous. Reformule ta réponse de manière claire.
Extraits : 
["Les règle du rugby La durée:la durée d'un match de rugby est de 2x 40minutes pour les séniors. les règles: il doit y avoir 15 joueurs obligatoirement dans une équipe de rugby. Un match commence par un coup d’envoi. Après le coup d’envoi, tout joueur qui est en jeu peut se saisir du ballon et courir en le portant. Un joueur peut lancer le ballon ou le botter.", 'Un joueur peut faire un touché à terre dans un en-but. Un porteur du ballon peut effectuer un raffut sur un adversaire. Toute action d’un joueur doit être faite dans le respect des Règles du Jeu. Un joueur et obliger de passer la balle en arrière Un terrain de rugby:', '• Hors-jeu :quand on et en avant du ballon. • Mêlée :huit joueurs de chaque équipe, liés entre eux sur trois lignes pour chaque équipe. • Pénalité :La pénalité est un geste technique. • Touch e: il faut lancer le ballons 